#### 고려대한국어대사전 오픈프로 포멧으로 변경 (proverb)

1. 오픈프로 필요 기타 칼럼 추가
2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
4. 설명 부분의 < > 부분 용법 내용으로 분류 
5. 품사 폰트 변경
6. "¶" 이걸 기준으로 설명과 예시 분류
7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다.
8. 용례 여러개일 경우 분류
9. <> [] 제거 만들기  
10. 뜻풀이 참조어 링크 태그로 변환
11. 중복제거
12. 네이버와 아이디 매칭

In [63]:
import pandas as pd
import re
from time import time

start_time = time()

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
symbols = ['①', '②', '③']

# input_file_path 부분을 채워주세요.
input_file_path = "../asset/proverb.csv"
target = pd.read_csv(input_file_path)

In [64]:
# 1. 오픈프로 필요한 기타 칼럼 추가 
target['type'] = '속담'
target['typediff'] = '괄호 없앤 속담/관용구'
target['shoulder'] = int(0)
target['onum'] = int(1)
target = target[['uid', 'eid', 'ohw', 'ocol', 'proverb', 'shoulder',
                'type', 'typediff', 'explan', 'onum']]

In [65]:
# 2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
LEN = len(target)
for i in range(LEN):
    target.loc[i, 'proverb'] = target.loc[i, 'proverb'][1:-1]

In [66]:
# 3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[0] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[1])
        target.loc[i, 'explan'] = temps[0][1:].strip()
        new_row = target.loc[i, :].copy()
        new_row['onum'] = 2
        new_row['explan'] = temps[1].strip()
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[2] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[2])
        target.loc[i, 'explan'] = temps[0].strip()
        new_row = target.loc[i, :].copy()
        new_row['explan'] = temps[1].strip()
        new_row['onum'] = 3
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

In [67]:
# 4. 설명 부분의 < > 부분 용법 내용으로 분류 
target['pattern'] = ''

LEN = len(target)
for i in range(LEN):
    p = r'\s*◀(.*?)▶' 
    m = re.search(p, target.loc[i, 'explan'])
    if m:
        result = m.group().strip() 
        target.loc[i, 'pattern'] = result[1:-1] 
        target.loc[i, 'explan'] = re.sub(p, '', target.loc[i, 'explan']).strip() 

In [68]:
# 5. 품사 폰트 변경 
# 보니까 proverb에는 pattern 정보가 없어 품사 폰트 변경 필요가 없다. 
# 칼럼 순서만 바꿔주자.
target = target[['uid', 'eid', 'ohw', 'ocol', 'proverb', 'shoulder', 'type', 'typediff', 'onum', 'pattern', 'explan']]

In [69]:
# 6. "¶" 이걸 기준으로 설명과 예시 분류
LEN = len(target)
target['explan'] = target['explan'].str.strip()
for i in range(LEN):
    temp = target.loc[i, 'explan'].split('¶')
    target.loc[i, 'explan'] = temp[0].strip()
    if len(temp) != 1:
        target.loc[i, 'example'] = temp[1].strip()
target['example'].fillna('', inplace=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_24370/4163738061.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['example'].fillna('', inplace=True)


In [70]:
# 7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다. (뜻번호당 하나만!!! 용례당 하나가 아님)
target['example'].fillna('', inplace=True)
target['similar_expression'] = ''

LEN = len(target)
for i in range(LEN):
    if '<유>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip() 

In [71]:
# 8. 용례 여러개일 경우 분류
LEN = len(target)
extra_list = []
for i in range(LEN):
    if '/' in target.loc[i, 'example']:
        temps = target.loc[i, 'example'].split(' / ')
        target.loc[i, 'example'] = temps[0].strip()
        for example in temps[1:]:
            new_row = target.loc[i, :].copy()
            new_row['example'] = temps[1].strip()
            extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 
                            'explan':[extra_list[i]['explan']], 'example':extra_list[i]['example']})
    target = pd.concat([target, new_row], ignore_index=True)

target['similar_expression'].fillna('', inplace=True)
target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_24370/1615679598.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['similar_expression'].fillna('', inplace=True)


In [72]:
# 9. 뜻풀이 참조어 링크 태그로 변환 + <유>

In [73]:
# 10. 중복제거
condition = target.duplicated(subset=['uid', 'onum'], keep='first')
condition2 = target.duplicated(subset=['uid'], keep='first')
target.loc[condition, 'uid':'explan'] = ""
target.loc[condition2, 'uid':'typediff'] = ""

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_24370/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_24370/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_24370/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[

In [74]:
# 11. 네이버와 아이디 매칭
target['naver_id'] = ""
target = target[['uid', 'naver_id', 'eid', 'ohw', 'ocol', 'proverb', 'shoulder', 'type', 'typediff',
                'onum', 'pattern', 'explan', 'example', 'similar_expression']]

ID = pd.read_csv('../asset/proverb_id.csv')
LEN = len(target)
LEN_ID = len(ID)
for i in range(LEN):
        target_proverb = target.loc[i, 'proverb']
        start = 0
        end = LEN_ID - 1
        while True:
                if start > end:
                        break
                mid = (start + end) // 2
                try_proverb = ID.loc[mid, 'proverb']
                if target_proverb == try_proverb:
                        target.loc[i, 'naver_id'] = ID.loc[mid, 'naver_id']
                        break
                elif target_proverb < try_proverb:
                        end = mid - 1
                else:
                        start = mid + 1

In [75]:
condition = (target['naver_id'] == "") & (target['uid'] != "")
# 신규어 개수 (매칭 안된 idiom)
print(f"신규(매칭 안됨) : {len(target[condition])}")

신규(매칭 안됨) : 38


In [76]:
target

,uid,naver_id,eid,ohw,ocol,proverb,shoulder,type,typediff,onum,pattern,explan,example,similar_expression
0,1,eeb837bbe99d4b81877e2d21fca6af0d,120,가갸,가갸뒤자도모른다,가갸 뒤 자〔뒷다리〕도 모른다,0,속담,괄호 없앤 속담/관용구,1,,글자를 모르는 사람을 비유적으로 이르는 말.,,
1,,,,,,,,,,2,,사리에 어두운 사람을 놀림조로 이르는 말.,가갸 뒤 자도 모르는 무지렁이를 사람으로 만들어 놨더니 이렇게 배신을 해?,"기역자 왼다리도 못 그린다, 낫 놓고 기역자도 모른다, 목불식정&EID=111624..."
2,2,f34c2f0cf1174684b907943d48492130,134,가게,가게기둥에입춘이라,가게 기둥에 입춘〔주련〕이라,0,속담,괄호 없앤 속담/관용구,1,,"보잘것없는 가겟집 기둥에 ‘입춘대길(立春大吉)’이라고 써붙인다는 뜻으로, 제격에 맞...",그렇게 비싸고 화려한 장식품이 누추한 내 방에 있는 것을 보면 사람들은 가게 기둥에...,"사모에 갓끈, 석새짚신에 구슬 감기, 재에 호 춤, 짚신에 국화 그리기, 초헌에 채찍질."
3,3,283b18ece16f47c493456696670987c7,435,가깝다,가까운남이먼친척보다낫다,가까운 남이 먼 친척〔일가〕보다 낫다,0,속담,괄호 없앤 속담/관용구,1,,이웃끼리 서로 친하게 지내면 먼 곳에 있는 일가보다 더 친하게 됨을 이르는 말.,가까운 남이 먼 친척보다 낫다더니 어젯밤에 갑자기 배가 너무 아팠는데 옆집 순이 엄...,지척의 원수가 천리의 벗보다 낫다.
4,4,d4fdd9d183404f5d83b1a64887ffd149,435,가깝다,가까운데를가도점심밥을싸가지고가거라,가까운 데를 가도 점심밥을 싸 가지고 가거라,0,속담,괄호 없앤 속담/관용구,1,,무슨 일에나 준비를 든든히 하여 실수가 없게 하라는 말. <유>십 리 길에 점심 싸기.,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8363,,,,,,,,,,2,,사람이 지나치게 야무지고 실속 차리기에 급급하여 사돈집 가을걷이를 하고 낟알을 털어...,,
8364,8156,,9710,갗,갗에서좀난다,갗에서 좀 난다,0,속담,괄호 없앤 속담/관용구,1,,"가죽에 좀이 나서 마침내 없어지면 결국 좀도 살 수 없게 된다는 뜻으로, 형제 간에...",,
8365,,,,,,,,,,2,,"가죽을 쏠아 먹는 좀이 가죽에서 생긴다는 뜻으로, 화근이 그 자체에 있음을 비유적으...",,
8366,8157,,143107,봉산,봉산수숫대같다,봉산 수숫대 같다,0,속담,괄호 없앤 속담/관용구,1,,‘키만 멀쑥하게 큰 사람’을 농담으로 이르는 말.,,


In [77]:
target.to_csv('../result/processed_proverb.csv', index=False)
end_time = time()
print(f"!!!FINISHED!!! 걸린시간 : {round(end_time - start_time, 2)}s")

!!!FINISHED!!! 걸린시간 : 2.78s
